In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
from skimage import transform
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd;
import numpy as np;
from torch.utils.data import Dataset, DataLoader
# from vis_utils import *
import random
import h5py
import math
import torch.nn.functional as F

In [2]:
# Hyperparameters
num_epochs = 2
num_classes = 10
batch_size = 1
learning_rate = 0.001

DATA_PATH = 'MNIST'
MODEL_STORE_PATH = 'pytorch_model\\'

In [3]:
# transforms to apply to the data
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

In [4]:
# transforms to apply to the data
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

In [5]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(16384, 1000)
#         self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 1)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
#         print('shape, before fc1', out.shape)
        out = self.fc1(out)
#         out = self.fc2(out)
        out = F.relu(self.fc2(out))
        return out

In [7]:
model = ConvNet()

# Loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()  # this is for regression mean squared loss

In [8]:
# # Train the model
# total_step = len(train_loader)
# loss_list = []
# acc_list = []
# for epoch in range(num_epochs):
#     for i, (images, labels) in enumerate(train_loader):
#         # Run the forward pass
#         print(images.shape)
#         images = Variable(images.float())
#         print(images.shape)
#         print(images.type)
#         labels = Variable(labels)
#         print('labels')
#         print(labels)
#         print(labels.shape)
#         outputs = model(images)
#         loss = loss_func(outputs, labels)
#         loss_list.append(loss.item())

#         # Backprop and perform Adam optimisation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Track the accuracy
#         total = labels.size(0)
#         _, predicted = torch.max(outputs.data, 1)
#         correct = (predicted == labels).sum().item()
#         acc_list.append(correct / total)

#         if (i + 1) % 100 == 0:
#             print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
#                   .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
#                           (correct / total) * 100))

In [9]:
# print('Here is the size of the input images:', images.shape)
# print('Here is the size of the outputs:', outputs.shape)

In [10]:
import matplotlib.pyplot as plt
from PIL import Image


# plt.imshow(IMAGE)
# plt.show()

In [11]:
number_brains_train = 100
brains = []
for i in range(1, 1+number_brains_train):
    with h5py.File('../Brains/%s.mat' %i, 'r') as f:
#         print(list(f.keys()))
#         print(list(f['cjdata'].keys()))
        brain = np.array(f['cjdata']['image'])
        brain = brain[::8, ::8]
        brains += [brain]

# columns = 2
# rows = 5

# fig=plt.figure(figsize=(20, 20))
# for i in range(1, 11):
#     brain = brains[i-1]
#     fig.add_subplot(rows, columns, i)
#     plt.imshow(brain)
# plt.show()


In [12]:
resulting_brains = []
vars_ = []
number_brains_blurried = 10

for l in range(len(brains)):
    test_brain = brains[l]/1000
    resulting_brains.append(test_brain)
    vars_.append(0)
    for i in range(number_brains_blurried):
        var = 0
        changed_brain = np.copy(test_brain)
        for j in range(64):
            for k in range(64):
                value = changed_brain[k][j]
                add = np.random.randint(max(-value, -1000+100*i), 1000-100*i)/1000
                #print(changed_brain[k][j])
                changed_brain[k][j] = value + add
                #print(changed_brain[k][j])

                var = var + add/1000
        resulting_brains.append(changed_brain)
        vars_.append(var)

In [13]:
print(len(vars_))
print(len(resulting_brains))

print((test_brain/1000)[32][32])

1100
1100
0.001454


In [14]:
# labels_ = np.random.randint(1, 10, 10).reshape(10, 1)
# print(np.array(brains).shape)
# brains = np.array(brains).reshape(10, 1, 64, 64)
# print(labels_.shape, brains.shape)
# print(labels_)

vars_ = np.array(vars_).reshape(number_brains_train*(number_brains_blurried+1), 1)
resulting_brains = np.array(resulting_brains).reshape(number_brains_train*(number_brains_blurried+1), 1, 64, 64)

print(vars_.shape[0], vars_.shape)

1100 (1100, 1)


In [15]:
# Train the model
total_step = vars_.shape[0]
loss_list = []
acc_list = []
for epoch in range(num_epochs):
#     for i, (images, labels) in enumerate(train_loader):
#         # Run the forward pass
#         images = Variable(images.float())
#         labels = Variable(labels)
#         outputs = model(images)
    for i in range(vars_.shape[0]):
        images = torch.from_numpy(resulting_brains[i].reshape(1, 1, 64, 64))
        images = Variable(images.float())

        labels = torch.from_numpy(vars_[i])
#         labels = Variable(labels.float())
#         labels = labels.float()
        outputs = model(images)
#         print('Out:', outputs)
#         print('Lab:', labels)
        loss = criterion(outputs, labels.float())
#         print('Loss:', loss)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

#         # Track the accuracy
#         total = labels.size(0)
#         _, predicted = torch.max(outputs.data, 1)
#         correct = (predicted == labels).sum().item()
#         acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/2], Step [100/1100], Loss: 0.3303
Epoch [1/2], Step [200/1100], Loss: 0.3243
Epoch [1/2], Step [300/1100], Loss: 0.1427
Epoch [1/2], Step [400/1100], Loss: 0.2779
Epoch [1/2], Step [500/1100], Loss: 0.0311
Epoch [1/2], Step [600/1100], Loss: 0.0177
Epoch [1/2], Step [700/1100], Loss: 0.0001
Epoch [1/2], Step [800/1100], Loss: 0.0130
Epoch [1/2], Step [900/1100], Loss: 0.0590
Epoch [1/2], Step [1000/1100], Loss: 0.0141
Epoch [1/2], Step [1100/1100], Loss: 0.0828
Epoch [2/2], Step [100/1100], Loss: 0.0936
Epoch [2/2], Step [200/1100], Loss: 0.0102
Epoch [2/2], Step [300/1100], Loss: 0.0977
Epoch [2/2], Step [400/1100], Loss: 0.0001
Epoch [2/2], Step [500/1100], Loss: 0.0419
Epoch [2/2], Step [600/1100], Loss: 0.0004
Epoch [2/2], Step [700/1100], Loss: 0.0025
Epoch [2/2], Step [800/1100], Loss: 0.0044
Epoch [2/2], Step [900/1100], Loss: 0.0005
Epoch [2/2], Step [1000/1100], Loss: 0.0014
Epoch [2/2], Step [1100/1100], Loss: 0.0637


In [16]:
resulting_brains_test = []
vars_test= []

brains_test = []
for i in range(len(brains), len(brains)+10):
    with h5py.File('../Brains/%s.mat' %i, 'r') as f:
#         print(list(f.keys()))
#         print(list(f['cjdata'].keys()))
        brain = np.array(f['cjdata']['image'])
        brain = brain[::8, ::8]
        brains_test += [brain]

for l in range(len(brains_test)):
    test_brain = brains_test[l]/1000
    resulting_brains_test.append(test_brain)
    vars_test.append(0)
    for i in range(10):
        var = 0
        changed_brain = np.copy(test_brain)
        for j in range(64):
            for k in range(64):
                value = changed_brain[k][j]
                add = np.random.randint(max(-value, -1000+100*i), 1000-100*i)/1000
                #print(changed_brain[k][j])
                changed_brain[k][j] = value + add
                #print(changed_brain[k][j])

                var = var + add/1000
        resulting_brains_test.append(changed_brain)
        vars_test.append(var)

In [17]:
# Test the model
model.eval()
with torch.no_grad():
#     for images, labels in test_loader:
   for i in range(len(vars_test)):
        images = torch.from_numpy(resulting_brains_test[i].reshape(1, 1, 64, 64))
        images = Variable(images.float())
        outputs = model(images)
          
        print('OUT', outputs.data[0][0], 'REAL', vars_test[i])

OUT tensor(0.1845) REAL 0
OUT tensor(1.7600) REAL 2.0207189999999913
OUT tensor(1.6139) REAL 1.851026000000002
OUT tensor(1.4118) REAL 1.6287540000000011
OUT tensor(1.2259) REAL 1.433189999999999
OUT tensor(1.0777) REAL 1.239981000000002
OUT tensor(0.9053) REAL 1.0180500000000006
OUT tensor(0.7174) REAL 0.8015870000000005
OUT tensor(0.5556) REAL 0.6094280000000017
OUT tensor(0.4003) REAL 0.40654499999999927
OUT tensor(0.2860) REAL 0.20621000000000128
OUT tensor(0.2098) REAL 0
OUT tensor(1.8171) REAL 2.041312
OUT tensor(1.6276) REAL 1.876362999999998
OUT tensor(1.4737) REAL 1.6220210000000044
OUT tensor(1.3128) REAL 1.4487199999999976
OUT tensor(1.1019) REAL 1.2390359999999985
OUT tensor(0.9288) REAL 1.0156530000000001
OUT tensor(0.7541) REAL 0.8209329999999966
OUT tensor(0.5915) REAL 0.6128819999999989
OUT tensor(0.4380) REAL 0.4119810000000003
OUT tensor(0.3093) REAL 0.19950200000000026
OUT tensor(0.3192) REAL 0
OUT tensor(1.9085) REAL 2.0448070000000036
OUT tensor(1.7279) REAL 1.8552

In [19]:
test_input = resulting_brains_test[5]
test_input = torch.from_numpy(test_input.reshape(1, 1, 64, 64))
# test_input.requires_grad = True
# print()
test_input = Variable(test_input.float(), requires_grad=True)
test_output = model(test_input)  
test_output.backward(retain_graph=True)
print(test_output)
# test_output = Variable(test_output, requires_grad=True)
# test_output.requires_grad = True
print(test_output)
print(test_input)
d = torch.autograd.grad(test_output, test_input, allow_unused=True)
print(d)

tensor([[1.0777]], grad_fn=<ReluBackward0>)
tensor([[1.0777]], grad_fn=<ReluBackward0>)
tensor([[[[0.1710, 0.0210, 0.0000,  ..., 0.1860, 0.2200, 0.3020],
          [0.5720, 0.3650, 0.1300,  ..., 0.0620, 0.3870, 0.5110],
          [0.0430, 0.3530, 0.1340,  ..., 0.5310, 0.1780, 0.3300],
          ...,
          [0.2400, 0.2280, 0.2010,  ..., 0.4570, 0.4160, 0.2390],
          [0.5950, 0.3330, 0.2310,  ..., 0.3310, 0.2910, 0.2740],
          [0.5640, 0.2370, 0.2720,  ..., 0.2010, 0.1550, 0.0530]]]],
       requires_grad=True)
(tensor([[[[ 4.0392e-04, -9.8434e-04,  6.6965e-04,  ...,  6.4405e-04,
           -8.0701e-04,  6.2893e-05],
          [ 1.2792e-03,  7.0771e-04, -5.3178e-04,  ..., -2.0317e-03,
            1.6572e-04,  5.3148e-04],
          [ 1.7241e-04, -1.4357e-03, -6.0469e-04,  ...,  2.4508e-03,
           -7.0479e-05,  2.0302e-03],
          ...,
          [ 8.4851e-04,  1.2267e-03, -1.7617e-03,  ...,  4.2838e-04,
            3.3253e-03,  1.8172e-03],
          [ 1.2884e-03,  1.

In [21]:
print(d[0].shape)

torch.Size([1, 1, 64, 64])


In [ ]:
# m = nn.Linear(2, 1)
# inputa = torch.ones(1, 2)
# inputa.requires_grad = True
# output = m(inputa).sum()
# output.backward(retain_graph=True)
# print(inputa.grad.data)

# print('INPUT:')
# print(inputa)
# print('OUTPUT:')
# print(output)
# print('LINEAR:')
# print(m)
# print('F(INPUT):', m(inputa))

In [ ]:
# print(m.weight)
# print(torch.autograd.grad(output, inputa))

In [ ]:
# test_input = resulting_brains_test[0]
# test_input = torch.from_numpy(test_input.reshape(1, 1, 64, 64))
# # test_input.requires_grad = True
# # print()
# test_input = Variable(test_input.float())
# test_output = model(test_input)  
# test_output.backward()
# print(test_output)
# # test_output.requires_grad = True

In [ ]:
print(test_input)

In [ ]:
d = torch.autograd.grad(test_output, test_input)